# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK
using AtomsBuilder

## Feeding an AtomsBase AbstractSystem to DFTK

In this example we construct a bulk silicon system using the `bulk` function
from [AtomsBuilder](https://github.com/JuliaMolSim/AtomsBuilder.jl). This function
uses tabulated data to set up a reasonable starting geometry and lattice for bulk silicon.

In [2]:
system = bulk(:Si)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


By default the atoms of an `AbstractSystem` employ the bare Coulomb potential.
To employ pseudpotential models (which is almost always advisable for
plane-wave DFT) one employs the `pseudopotential` keyword argument in
model constructors such as `model_DFT`.
For example we can employ a `PseudoFamily` object
from the [PseudoPotentialData](https://github.com/JuliaMolSim/PseudoPotentialData.jl)
package. See its documentation for more information on the available
pseudopotential families and how to select them.

In [3]:
using PseudoPotentialData  # defines PseudoFamily

pd_lda_family = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
model = model_DFT(system; functionals=LDA(), temperature=1e-3,
                  pseudopotentials=pd_lda_family)

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [0         , 5.13061   , 5.13061   ]
                           [5.13061   , 0         , 5.13061   ]
                           [5.13061   , 5.13061   , 0         ]
    unit cell volume     : 270.11 Bohr³

    atoms                : Si₂
    pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")

    num. electrons       : 8
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                           Entropy()

Alternatively the `pseudopotentials` object also accepts a `Dict{Symbol,String}`,
which provides for each element symbol the filename or identifier
of the pseudopotential to be employed, e.g.

In [4]:
path_to_pspfile = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")[:Si]
model = model_DFT(system; functionals=LDA(), temperature=1e-3,
                  pseudopotentials=Dict(:Si => path_to_pspfile))

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [0         , 5.13061   , 5.13061   ]
                           [5.13061   , 0         , 5.13061   ]
                           [5.13061   , 5.13061   , 0         ]
    unit cell volume     : 270.11 Bohr³

    atoms                : Si₂
    atom potentials      : ElementPsp(:Si, "/home/runner/.julia/artifacts/966fd9cdcd7dbaba6dc2bf43ee50dd81e63e8837/Si.gth")
                           ElementPsp(:Si, "/home/runner/.julia/artifacts/966fd9cdcd7dbaba6dc2bf43ee50dd81e63e8837/Si.gth")

    num. electrons       : 8
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                 

We can then discretise such a model and solve:

In [5]:
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921708193518                   -0.69    5.5    202ms
  2   -7.926133120856       -2.35       -1.22    1.0    218ms
  3   -7.926833183563       -3.15       -2.37    2.0    239ms
  4   -7.926861263207       -4.55       -2.99    3.0    236ms
  5   -7.926861642857       -6.42       -3.38    2.0    173ms
  6   -7.926861669249       -7.58       -3.81    1.4    162ms
  7   -7.926861679442       -7.99       -4.17    1.8    160ms
  8   -7.926861681774       -8.63       -5.12    1.2    157ms
  9   -7.926861681863      -10.05       -5.25    2.5    189ms
 10   -7.926861681869      -11.20       -5.61    1.1    154ms
 11   -7.926861681873      -11.51       -6.54    1.0    153ms
 12   -7.926861681873      -13.27       -6.95    2.5    184ms
 13   -7.926861681873      -14.45       -7.43    1.8    167ms
 14   -7.926861681873      -15.05       -7.75    1.6    165ms
 15   -7.

If we did not want to use
[AtomsBuilder](https://github.com/JuliaMolSim/AtomsBuilder.jl)
we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
which directly yields an AbstractSystem.

In [6]:
using AtomsIO
system = load_system("Si.extxyz");

Run the LDA calculation:

In [7]:
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(system; pseudopotentials, functionals=LDA(), temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921723191649                   -0.69    5.5    203ms
  2   -7.926131181675       -2.36       -1.22    1.0    190ms
  3   -7.926832980810       -3.15       -2.37    2.0    169ms
  4   -7.926861371682       -4.55       -2.98    3.2    231ms
  5   -7.926861644644       -6.56       -3.37    1.8    166ms
  6   -7.926861670342       -7.59       -3.84    1.6    156ms
  7   -7.926861679883       -8.02       -4.24    1.6    239ms
  8   -7.926861681763       -8.73       -4.93    1.4    916ms
  9   -7.926861681861      -10.01       -5.24    2.4    177ms
 10   -7.926861681871      -11.00       -5.90    1.2    151ms
 11   -7.926861681872      -11.88       -6.32    2.2    176ms
 12   -7.926861681873      -12.96       -7.69    1.8    163ms
 13   -7.926861681873      -14.35       -7.67    3.6    219ms
 14   -7.926861681873   +    -Inf       -8.14    1.0    147ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [8]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(system; pseudopotentials, functionals=LDA(), temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921726695948                   -0.69    5.6    228ms
  2   -7.926134252582       -2.36       -1.22    1.0    129ms
  3   -7.926836170092       -3.15       -2.37    2.0    152ms
  4   -7.926864731581       -4.54       -2.99    3.0    227ms
  5   -7.926865058206       -6.49       -3.38    2.1    158ms
  6   -7.926865082045       -7.62       -3.86    1.4    133ms
  7   -7.926865090609       -8.07       -4.18    1.5    137ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [9]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [10]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, pseudopotentials)
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
